<a href="https://colab.research.google.com/github/rameshkn/sharing-github/blob/master/1DCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

from Data import fetch_1d_data
from random import shuffle

batch_size = 65536
epochs = 28


with tf.device("/gpu:0"):
    mean_conv1 = 0.5
    mean_conv2 = 0.4
    mean_conv3 = 0.35
    mean_conv4 = 0.3

    sd_conv1 = 0.45
    sd_conv2 = 0.35
    sd_conv3 = 0.25
    sd_conv4 = 0.20

    mode = tf.placeholder(dtype=tf.bool, name='mode_of_operation')
    input_layer = tf.placeholder(dtype=tf.float32, shape=(batch_size, 9), name='input_images')
    input_layer1 = tf.reshape(input_layer, [batch_size, 9, 1])
    labels = tf.placeholder(dtype=tf.float32, shape=(batch_size, 1), name='gt_labels')
    global_step = tf.Variable(0.0, trainable=False)

    # 1st convolution layer
    he_init_conv1 = tf.truncated_normal_initializer(mean=mean_conv1, stddev=sd_conv1)

    conv1 = tf.layers.conv1d(inputs=input_layer1,
                             kernel_size=3,
                             filters=64,
                             strides=1,
                             padding='same',
                             use_bias=True,
                             activation=None,
                             kernel_initializer=he_init_conv1,
                             name='conv1')

    '''m_1, v_1 = tf.nn.moments(x=conv1, axes=[0], keep_dims=False)
    off_1 = tf.Variable(tf.zeros_like(m_1), name='off_1')
    scale_1 = tf.Variable(tf.ones_like(m_1), name='scale_1')

    bn_1 = tf.nn.batch_normalization(x=conv1, mean=m_1, variance=v_1, offset=off_1, scale=scale_1,
                                     variance_epsilon=1e-1, name='bn1')'''

    bn_1 = tf.layers.batch_normalization(inputs=conv1, momentum=0.9, axis=-1, training=mode, name='bn1')
    conv1_bn = tf.nn.relu(bn_1, name='layer_1')

    # 2nd convolutional layer
    he_init_conv2 = tf.truncated_normal_initializer(mean=mean_conv2, stddev=sd_conv2)

    conv2 = tf.layers.conv1d(inputs=conv1_bn,
                             kernel_size=3,
                             filters=64,
                             strides=1,
                             padding='same',
                             use_bias=True,
                             activation=None,
                             kernel_initializer=he_init_conv2,
                             name='conv2')

    '''m_2, v_2 = tf.nn.moments(x=conv2, axes=[0], keep_dims=False)
    off_2 = tf.Variable(tf.zeros_like(m_2))
    scale_2 = tf.Variable(tf.ones_like(m_2))

    bn_2 = tf.nn.batch_normalization(x=conv2, mean=m_2, variance=v_2, offset=off_2, scale=scale_2,
                                     variance_epsilon=1e-5, name='bn2')'''

    bn_2 = tf.layers.batch_normalization(inputs=conv2, momentum=0.9, axis=-1, training=mode, name='bn2')
    conv2_bn = tf.nn.relu(bn_2, name='layer_2')

    # 3rd convolutional layer
    he_init_conv3 = tf.truncated_normal_initializer(mean=mean_conv3, stddev=sd_conv3)

    conv3 = tf.layers.conv1d(inputs=conv2_bn,
                             kernel_size=3,
                             filters=64,
                             strides=1,
                             padding='same',
                             use_bias=True,
                             activation=None,
                             kernel_initializer=he_init_conv3,
                             name='conv3')

    '''m_3, v_3 = tf.nn.moments(x=conv3, axes=[0], keep_dims=False)
    off_3 = tf.Variable(tf.zeros_like(m_3))
    scale_3 = tf.Variable(tf.ones_like(m_3))

    bn_3 = tf.nn.batch_normalization(x=conv3, mean=m_3, variance=v_3, offset=off_3, scale=scale_3,
                                     variance_epsilon=1e-5, name='bn3')'''

    bn_3 = tf.layers.batch_normalization(inputs=conv3, momentum=0.9, axis=-1, training=mode, name='bn3')
    conv3_bn = tf.nn.relu(bn_3, name='layer_3')

    he_init_conv4 = tf.truncated_normal_initializer(mean=mean_conv4, stddev=sd_conv4)

    conv4 = tf.layers.conv1d(inputs=conv3_bn,
                             kernel_size=3,
                             filters=64,
                             strides=1,
                             padding='same',
                             use_bias=True,
                             activation=None,
                             kernel_initializer=he_init_conv4,
                             name='conv4')

    bn_4 = tf.layers.batch_normalization(inputs=conv4, momentum=0.9, axis=-1, training=mode, name='bn4')
    conv4_bn = tf.nn.relu(bn_4, name='layer_4')

    shape = tf.shape(conv4_bn)
    conv4_re = tf.reshape(conv4_bn, [batch_size, shape[1]*shape[2]])

    dense1 = tf.layers.dense(inputs=conv4_re, units=196)
    bn_5 = tf.layers.batch_normalization(inputs=dense1, momentum=0.9, axis=-1, training=mode, name='bn5')
    dense1_act = tf.nn.relu(bn_5, name='dense_1')

    dropout = tf.layers.dropout(inputs=dense1_act, rate=0.2, training=mode)
    predict = tf.layers.dense(inputs=dropout, units=1, name='predict')

    loss = tf.losses.mean_squared_error(labels, predict)

    lr = tf.train.exponential_decay(9e-4, global_step, 15000, 1.5, staircase=True)
    optimizer = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.9)

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_op = optimizer.minimize(loss=loss,
                                      global_step=global_step)

    predictions = tf.add(predict, tf.zeros_like(predict), name='prediction')

saver = tf.train.Saver()
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC'


with tf.Session(config=config) as sess:
    # Load training and eval data
    train_dir_path = 'train'
    train_data, train_label = fetch_1d_data(train_dir_path)

    train_data = np.array(train_data, dtype=np.float32)
    train_label = np.array(train_label, dtype=np.int32)

    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())

    for i in range(epochs):
        index = np.arange(0, train_data.shape[0] - batch_size)
        shuffle(index)
        for t in range(0, len(index) - batch_size, batch_size):
            data = {input_layer: train_data[index[t:t + batch_size]],
                    labels: train_label[index[t:t + batch_size]],
     